## 3 Assignment
## 다음 3개의 문장을 batch data로 활용해 RNN을 학습해보자 (아래의 미완성 코드, 위 실습코드, 실행결과를 참고)

* 'howareyou'
* 'whats up?'
* 'iamgreat.'

In [71]:
# import
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# for reproducibility
torch.manual_seed(100)

In [72]:
# Dictionary. 샘플 문장에 쓰인 글자 수가 모두 9로 같다.
sample_sentences = ['howareyou', 'whats up?', 'iamgreat.']
char_set = list(set(''.join(sample_sentences)))
dic = {c: i for i, c in enumerate(char_set)}

print(sample_sentences[0])
print(len(sample_sentences[0]))
print(sample_sentences[1])
print(len(sample_sentences[1]))
print(sample_sentences[2])
print(len(sample_sentences[2]))

howareyou
9
whats up?
9
iamgreat.
9


In [73]:
# 샘플 문장의 수는 세개, 16개의 문자가 char_set에 입력되며, 그 중에는 ' '도 포함된다.
print(sample_sentences)
print(char_set)
print(dic)

['howareyou', 'whats up?', 'iamgreat.']
['t', 's', 'i', 'e', 'r', ' ', 'h', 'm', 'a', 'u', '.', 'p', 'o', 'y', '?', 'w', 'g']
{'t': 0, 's': 1, 'i': 2, 'e': 3, 'r': 4, ' ': 5, 'h': 6, 'm': 7, 'a': 8, 'u': 9, '.': 10, 'p': 11, 'o': 12, 'y': 13, '?': 14, 'w': 15, 'g': 16}


In [74]:
# Parameters. Input으로 17개의 글자가 들어감으로 17개 ,hidden size = Output Size이기 때문에 글자의 갯수인 17개가 맞다.
# 결국 sample_sentences의 문자들이 인풋과 아웃풋의 후보가 되므로, sample_sentences에 쓰인 문자의 갯수인 len(dic)이 결국 인풋과 아웃풋의 사이즈가 되는 것이다.
dic_size = len(dic)
input_size = dic_size
hidden_size = dic_size

In [75]:
# 딕셔너리 형으로 정의된 dic의 크기. dic은 char_set을 key로 사용하고, 숫자는 char_set의 순서에 따라 매긴 것이므로 dic과 char_set의 크기는 같다.
print(dic_size)
print(len(char_set))

17
17


In [76]:
# Dataset setting. 인풋 batch, 타겟 batch 
input_batch = []
target_batch = []

In [77]:
# 각 sentence를 학습시킬 때 for문을 사용하여 순차적으로 howareyou부터 iamgreat까지 순차적으로 학습한다.
# 학습시킬 sentence의 차례에는 x_data에 sentence, x_one_hot에 원핫코드, y_data에 정답을 입력한다.
# 따라서 모델의 input으로 들어갈 수 있는 글자는 sentence에 포함되어있는 글자이다.
# 또한 모델의 output으로 나올 수 있는 글자도 sentence에 포함되어있는 글자가 되며 이는 sentence의 두번째 글자 부터이다.
for sentence in sample_sentences:
  x_data = [dic[c] for c in sample_sentences[sample_sentences.index(sentence)][:-1]]
  x_one_hot = [np.eye(dic_size)[x] for x in x_data]
  y_data = [dic[c] for c in sample_sentences[sample_sentences.index(sentence)][1:]]

# x_one_hot, y_data를 batch(List형)에 넣은 후 Tensor로 변환하는 과정. 모델의 학습은 Tensor를 통해 이루어진다.
  input_batch.append(x_one_hot)
  target_batch.append(y_data)

  IpTs = torch.FloatTensor(input_batch)
  tgTs = torch.LongTensor(target_batch)

  # Model. 강의 자료에 있는 값이 0.05, 500으로 고정. 더 원활한 학습을 위해서는 파라미터의 수정이 필요하다.
  learning_rate = 0.05
  training_epochs = 500
  model = nn.RNN(input_size, hidden_size, batch_first=True)

  # define cost/loss & optimizer (분류 과정을 거치므로 CrossEntropyLoss 함수의 도움을 받을 수 있다. Adam은 SGD방식과 유사하지만 더 진화한 알고리즘을 가진다.)
  criterion = nn.CrossEntropyLoss()    # Softmax
  optimizer = optim.Adam(model.parameters(), lr=learning_rate)

  # train
  for epoch in range(training_epochs):
    optimizer.zero_grad()
    outputs, _status = model(IpTs)
    loss = criterion(outputs.reshape(-1, dic_size), tgTs.reshape(-1))
    loss.backward()
    optimizer.step()
    if epoch %100 == 99:
      result = outputs.data.numpy().argmax(axis=2)
for sentence in result:
        print(''.join([char_set[c] for c in np.squeeze(sentence)]))


owareyou
hows up?
amgreat.
